In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import shap

In [2]:


# ==========================================
# CARGA DE DATOS
# ==========================================
app_train = pd.read_csv('../data/interim/aplicationtrainlimpio2.csv') #changed the new cleaned dataset
app_test = pd.read_csv('../data/raw/application_ts.csv')
bureau = pd.read_csv('../data/raw/bureau.csv')
bureau_balance = pd.read_csv('../data/raw/bureau_balance.csv')
previous = pd.read_csv('../data/raw/previous_application.csv')
pos_cash = pd.read_csv('../data/raw/POS_CASH_balance.csv')
credit_card = pd.read_csv('../data/raw/credit_card_balance.csv')
installments = pd.read_csv('../data/raw/installments_payments.csv')

# ==========================================
# FUNCIÓN DE DECAY EXPONENCIAL
# ==========================================
def exponential_decay(days, half_life=365):
    """
    Aplica decay exponencial. half_life en días.
    días=-30 (hace 1 mes) → peso alto
    días=-730 (hace 2 años) → peso bajo
    """
    return np.exp(days / half_life)

# ==========================================
# BUREAU FEATURES (ORIGINAL + TEMPORAL)
# ==========================================
print("Procesando Bureau...")

bureau_full = bureau.merge(bureau_balance, on="SK_ID_BUREAU", how="left")

# Features originales
bureau_features = (
    bureau_full
    .groupby("SK_ID_CURR")
    .agg(
        bureau_loans=("SK_ID_BUREAU", "nunique"),
        bureau_days_credit_mean=("DAYS_CREDIT", "mean"),
        bureau_days_credit_min=("DAYS_CREDIT", "min"),
        bureau_credit_sum=("AMT_CREDIT_SUM", "sum"),
        bureau_credit_active=("CREDIT_ACTIVE", lambda x: (x == "Active").sum()),
        bureau_months_reported=("MONTHS_BALANCE", "count")
    )
    .reset_index()
)

active_bureau = bureau[bureau["CREDIT_ACTIVE"] == "Active"].groupby("SK_ID_CURR").agg(
    active_loans_count=("SK_ID_BUREAU", "count"),
    active_debt_sum=("AMT_CREDIT_SUM_DEBT", "sum"),
    active_overdue_sum=("AMT_CREDIT_SUM_OVERDUE", "sum")
).reset_index()

recent_bureau = bureau[bureau["DAYS_CREDIT"] >= -730].groupby("SK_ID_CURR").agg(
    recent_loans_count=("SK_ID_BUREAU", "count"),
    recent_overdue_mean=("AMT_CREDIT_SUM_OVERDUE", "mean")
).reset_index()

bureau_features = bureau_features.merge(active_bureau, on="SK_ID_CURR", how="left")
bureau_features = bureau_features.merge(recent_bureau, on="SK_ID_CURR", how="left").fillna(0)

# 🆕 FEATURES TEMPORALES CON DECAY
bureau['weight'] = exponential_decay(bureau['DAYS_CREDIT'], half_life=365)

bureau_weighted = bureau.groupby('SK_ID_CURR', group_keys=False).apply(
    lambda x: pd.Series({
        'bureau_weighted_credit': (x['AMT_CREDIT_SUM'] * x['weight']).sum() / (x['weight'].sum() + 1e-5),
        'bureau_weighted_debt': (x['AMT_CREDIT_SUM_DEBT'].fillna(0) * x['weight']).sum() / (x['weight'].sum() + 1e-5),
        'bureau_weighted_overdue': (x['AMT_CREDIT_SUM_OVERDUE'].fillna(0) * x['weight']).sum() / (x['weight'].sum() + 1e-5),
        'bureau_weighted_active_ratio': ((x['CREDIT_ACTIVE'] == 'Active').astype(int) * x['weight']).sum() / (x['weight'].sum() + 1e-5)
    }), include_groups=False
).reset_index()

bureau_recent_vs_old = bureau.groupby('SK_ID_CURR', group_keys=False).apply(
    lambda x: pd.Series({
        'bureau_recent_vs_old_overdue': (
            x[x['DAYS_CREDIT'] >= -365]['AMT_CREDIT_SUM_OVERDUE'].fillna(0).mean() /
            (x[x['DAYS_CREDIT'] < -365]['AMT_CREDIT_SUM_OVERDUE'].fillna(0).mean() + 1)
        ),
        'bureau_recent_vs_old_debt': (
            x[x['DAYS_CREDIT'] >= -365]['AMT_CREDIT_SUM_DEBT'].fillna(0).sum() /
            (x[x['DAYS_CREDIT'] < -365]['AMT_CREDIT_SUM_DEBT'].fillna(0).sum() + 1)
        )
    }), include_groups=False
).reset_index()

bureau_features = bureau_features.merge(bureau_weighted, on='SK_ID_CURR', how='left')
bureau_features = bureau_features.merge(bureau_recent_vs_old, on='SK_ID_CURR', how='left')

# ==========================================
# PREVIOUS APPLICATIONS (ORIGINAL + TEMPORAL)
# ==========================================
print("Procesando Previous Applications...")

prev_features = (
    previous
    .groupby("SK_ID_CURR")
    .agg(
        prev_apps=("SK_ID_PREV", "nunique"),
        prev_amt_mean=("AMT_APPLICATION", "mean"),
        prev_amt_max=("AMT_APPLICATION", "max"),
        prev_refused=("NAME_CONTRACT_STATUS", lambda x: (x == "Refused").sum()),
        prev_approved=("NAME_CONTRACT_STATUS", lambda x: (x == "Approved").sum()),
        prev_days_decision_mean=("DAYS_DECISION", "mean")
    )
    .reset_index()
)

# 🆕 FEATURES TEMPORALES
previous['weight'] = exponential_decay(previous['DAYS_DECISION'], half_life=365)

prev_weighted = previous.groupby('SK_ID_CURR', group_keys=False).apply(
    lambda x: pd.Series({
        'prev_weighted_amt': (x['AMT_APPLICATION'] * x['weight']).sum() / (x['weight'].sum() + 1e-5),
        'prev_weighted_refused_ratio': ((x['NAME_CONTRACT_STATUS'] == 'Refused').astype(int) * x['weight']).sum() / (x['weight'].sum() + 1e-5),
        'prev_recent_refused_count': (x[x['DAYS_DECISION'] >= -365]['NAME_CONTRACT_STATUS'] == 'Refused').sum(),
        'prev_recent_approved_count': (x[x['DAYS_DECISION'] >= -365]['NAME_CONTRACT_STATUS'] == 'Approved').sum()
    }), include_groups=False
).reset_index()

prev_features = prev_features.merge(prev_weighted, on='SK_ID_CURR', how='left')

# ==========================================
# INSTALLMENTS (ORIGINAL + TEMPORAL)
# ==========================================
print("Procesando Installments...")

installments["late"] = (installments["DAYS_ENTRY_PAYMENT"] > installments["DAYS_INSTALMENT"]).astype(int)
installments["dbd"] = installments["DAYS_INSTALMENT"] - installments["DAYS_ENTRY_PAYMENT"]

inst_features = installments.groupby("SK_ID_CURR").agg(
    inst_count_total=("SK_ID_PREV", "count"),
    inst_late_ratio_total=("late", "mean"),
    inst_dbd_mean_total=("dbd", "mean")
).reset_index()

recent_inst = installments[installments["DAYS_INSTALMENT"] >= -365].groupby("SK_ID_CURR").agg(
    inst_late_ratio_1y=("late", "mean"),
    inst_dbd_mean_1y=("dbd", "mean"),
    inst_amt_paid_1y=("AMT_PAYMENT", "sum")
).reset_index()

inst_features = inst_features.merge(recent_inst, on="SK_ID_CURR", how="left").fillna(0)
inst_features['inst_late_trend'] = inst_features['inst_late_ratio_1y'] / (inst_features['inst_late_ratio_total'] + 0.001)

# 🆕 FEATURES TEMPORALES POR PERÍODOS
installments['period'] = pd.cut(
    installments['DAYS_INSTALMENT'],
    bins=[-np.inf, -365, -180, 0],
    labels=['old', 'mid', 'recent']
)

inst_temporal = installments.groupby(['SK_ID_CURR', 'period'])['late'].mean().unstack(fill_value=0)
inst_temporal.columns = [f'inst_late_{col}' for col in inst_temporal.columns]
inst_temporal = inst_temporal.reset_index()
inst_temporal['inst_late_deterioration'] = (
    inst_temporal['inst_late_recent'] - inst_temporal['inst_late_old']
)

inst_features = inst_features.merge(inst_temporal, on='SK_ID_CURR', how='left')

# ==========================================
# POS_CASH (ORIGINAL + TEMPORAL)
# ==========================================
print("Procesando POS Cash...")

pos_features = (
    pos_cash
    .groupby("SK_ID_CURR")
    .agg(
        pos_loans=("SK_ID_PREV", "nunique"),
        pos_months=("MONTHS_BALANCE", "count"),
        pos_dpd_mean=("SK_DPD", "mean"),
        pos_dpd_def_mean=("SK_DPD_DEF", "mean")
    )
    .reset_index()
)

pos_recent = pos_cash[pos_cash["MONTHS_BALANCE"] >= -6].groupby("SK_ID_CURR").agg(
    pos_recent_max_dpd=("SK_DPD", "max"),
    pos_recent_count_dpd=("SK_DPD", lambda x: (x > 0).sum())
).reset_index()

pos_features = pos_features.merge(pos_recent, on="SK_ID_CURR", how="left").fillna(0)

# 🆕 FEATURES TEMPORALES MÚLTIPLES PERÍODOS
for months in [3, 6, 12]:
    pos_period = pos_cash[pos_cash['MONTHS_BALANCE'] >= -months].groupby('SK_ID_CURR').agg(
        dpd_mean=('SK_DPD', 'mean'),
        dpd_max=('SK_DPD', 'max'),
        dpd_std=('SK_DPD', 'std')
    )
    pos_period.columns = [f'pos_dpd_{stat}_{months}m' for stat in ['mean', 'max', 'std']]
    pos_features = pos_features.merge(pos_period.reset_index(), on='SK_ID_CURR', how='left')

pos_accel = pos_cash.groupby('SK_ID_CURR', group_keys=False).apply(
    lambda x: pd.Series({
        'pos_dpd_acceleration': (
            x[x['MONTHS_BALANCE'] >= -3]['SK_DPD'].mean() -
            x[(x['MONTHS_BALANCE'] >= -12) & (x['MONTHS_BALANCE'] < -3)]['SK_DPD'].mean()
        )
    }), include_groups=False
).reset_index()

pos_features = pos_features.merge(pos_accel, on='SK_ID_CURR', how='left')

# ==========================================
# CREDIT CARD (ORIGINAL + TEMPORAL)
# ==========================================
print("Procesando Credit Card...")

cc_features = (
    credit_card
    .groupby("SK_ID_CURR")
    .agg(
        cc_loans=("SK_ID_PREV", "nunique"),
        cc_balance_mean=("AMT_BALANCE", "mean"),
        cc_limit_mean=("AMT_CREDIT_LIMIT_ACTUAL", "mean"),
        cc_utilization=("AMT_BALANCE", "mean")
    )
    .reset_index()
)

# 🆕 FEATURES TEMPORALES
cc_temporal = credit_card.groupby('SK_ID_CURR', group_keys=False).apply(
    lambda x: pd.Series({
        'cc_util_recent': (
            x[x['MONTHS_BALANCE'] >= -3]['AMT_BALANCE'].mean() /
            (x[x['MONTHS_BALANCE'] >= -3]['AMT_CREDIT_LIMIT_ACTUAL'].mean() + 1)
        ),
        'cc_util_historical': (
            x['AMT_BALANCE'].mean() / (x['AMT_CREDIT_LIMIT_ACTUAL'].mean() + 1)
        ),
        'cc_balance_trend': (
            x[x['MONTHS_BALANCE'] >= -3]['AMT_BALANCE'].mean() -
            x[x['MONTHS_BALANCE'] < -3]['AMT_BALANCE'].mean()
        )
    }), include_groups=False
).reset_index()

cc_temporal['cc_util_deterioration'] = cc_temporal['cc_util_recent'] - cc_temporal['cc_util_historical']
cc_features = cc_features.merge(cc_temporal, on='SK_ID_CURR', how='left')

# ==========================================
# MERGE FINAL
# ==========================================
print("Uniendo todas las features...")

df_train = (
    app_train
    .merge(bureau_features, on="SK_ID_CURR", how="left")
    .merge(prev_features, on="SK_ID_CURR", how="left")
    .merge(inst_features, on="SK_ID_CURR", how="left")
    .merge(pos_features, on="SK_ID_CURR", how="left")
    .merge(cc_features, on="SK_ID_CURR", how="left")
)

# ==========================================
# 🆕 FEATURES CROSS-TABLE
# ==========================================
print("Generando features combinadas...")

df_train['debt_to_income_recent'] = (
    df_train['bureau_weighted_debt'] / (df_train['AMT_INCOME_TOTAL'] + 1)
)

df_train['monthly_payment_burden'] = (
    (df_train['AMT_ANNUITY'].fillna(0) + df_train['inst_amt_paid_1y'] / 12) /
    (df_train['AMT_INCOME_TOTAL'] / 12 + 1)
)

df_train['combined_deterioration_score'] = (
    df_train['inst_late_deterioration'].fillna(0) +
    df_train['pos_dpd_acceleration'].fillna(0) +
    df_train['cc_util_deterioration'].fillna(0) +
    df_train['bureau_recent_vs_old_overdue'].fillna(1) - 1
)

# ==========================================
# GUARDAR RESULTADO
# ==========================================
df_train.to_parquet("../data/interim/train_final_with_temporal2.parquet")

print("\n" + "="*50)
print("✅ PROCESO COMPLETADO")
print("="*50)
print(f"Total de columnas: {df_train.shape[1]}")
print(f"Total de filas: {df_train.shape[0]}")
print(f"\nNuevas features temporales añadidas:")
print(f"  - Bureau: weighted features + ratios reciente vs antiguo")
print(f"  - Previous: weighted features + conteos recientes")
print(f"  - Installments: deterioration por períodos")
print(f"  - POS Cash: aceleración DPD + stats por 3/6/12 meses")
print(f"  - Credit Card: utilización reciente vs histórica")
print(f"  - Cross-features: debt_to_income, payment_burden, deterioration_score")
print(f"\nArchivo guardado: ../data/interim/train_final_with_temporal2.parquet")

# Mostrar info del dataframe
print("\n" + "="*50)
df_train.info()
print("\n")
df_train.head()

Procesando Bureau...
Procesando Previous Applications...
Procesando Installments...


C:\Users\usuario\AppData\Local\Temp\ipykernel_1832\1245569772.py:151: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  inst_temporal = installments.groupby(['SK_ID_CURR', 'period'])['late'].mean().unstack(fill_value=0)


Procesando POS Cash...
Procesando Credit Card...
Uniendo todas las features...
Generando features combinadas...

✅ PROCESO COMPLETADO
Total de columnas: 113
Total de filas: 307511

Nuevas features temporales añadidas:
  - Bureau: weighted features + ratios reciente vs antiguo
  - Previous: weighted features + conteos recientes
  - Installments: deterioration por períodos
  - POS Cash: aceleración DPD + stats por 3/6/12 meses
  - Credit Card: utilización reciente vs histórica
  - Cross-features: debt_to_income, payment_burden, deterioration_score

Archivo guardado: ../data/interim/train_final_with_temporal2.parquet

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 113 entries, SK_ID_CURR to combined_deterioration_score
dtypes: float64(87), int64(14), object(12)
memory usage: 265.1+ MB




,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,cc_balance_mean,cc_limit_mean,cc_utilization,cc_util_recent,cc_util_historical,cc_balance_trend,cc_util_deterioration,debt_to_income_recent,monthly_payment_burden,combined_deterioration_score
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.049099,2.228358,-1.000000
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.586529,0.000000
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.199787,0.000000
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,270000.0,0.0,0.0,0.0,0.0,0.0,NaN,9.912453,0.000000
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,3.875204,-0.301887


In [3]:
# ==========================================
# AGREGACIONES AVANZADAS PARA MEJORAR MÉTRICAS
# ==========================================

# ==========================================
# 1. BUREAU - FEATURES AVANZADAS
# ==========================================
print("🔥 Generando features avanzadas de Bureau...")

# Análisis de comportamiento por tipo de crédito
bureau_by_type = bureau.groupby(['SK_ID_CURR', 'CREDIT_TYPE'], group_keys=False).agg(
    count=('SK_ID_BUREAU', 'count'),
    avg_credit=('AMT_CREDIT_SUM', 'mean'),
    max_overdue=('AMT_CREDIT_SUM_OVERDUE', 'max')
).reset_index()

bureau_type_pivot = bureau_by_type.pivot_table(
    index='SK_ID_CURR',
    columns='CREDIT_TYPE',
    values='count',
    fill_value=0
).reset_index()
bureau_type_pivot.columns = ['SK_ID_CURR'] + [f'bureau_type_{col}' for col in bureau_type_pivot.columns[1:]]

# Diversidad de créditos (Shannon entropy)
bureau_diversity = bureau.groupby('SK_ID_CURR')['CREDIT_TYPE'].apply(
    lambda x: -sum((x.value_counts(normalize=True) * np.log(x.value_counts(normalize=True) + 1e-10)))
).reset_index()
bureau_diversity.columns = ['SK_ID_CURR', 'bureau_credit_diversity']

# Ratios de deuda críticos
bureau_ratios = bureau.groupby('SK_ID_CURR', group_keys=False).apply(
    lambda x: pd.Series({
        'bureau_debt_to_credit_ratio': x['AMT_CREDIT_SUM_DEBT'].sum() / (x['AMT_CREDIT_SUM'].sum() + 1),
        'bureau_overdue_to_debt_ratio': x['AMT_CREDIT_SUM_OVERDUE'].sum() / (x['AMT_CREDIT_SUM_DEBT'].sum() + 1),
        'bureau_max_overdue_to_income': x['AMT_CREDIT_SUM_OVERDUE'].max(),  # Se combinará después con income
        'bureau_prolongation_count': (x['CNT_CREDIT_PROLONG'] > 0).sum(),
        'bureau_prolongation_ratio': (x['CNT_CREDIT_PROLONG'] > 0).mean()
    }), include_groups=False
).reset_index()

# Análisis temporal refinado
bureau_temporal_advanced = bureau.groupby('SK_ID_CURR', group_keys=False).apply(
    lambda x: pd.Series({
        # Ventanas temporales específicas
        'bureau_overdue_last_3m': x[x['DAYS_CREDIT'] >= -90]['AMT_CREDIT_SUM_OVERDUE'].mean(),
        'bureau_overdue_last_6m': x[x['DAYS_CREDIT'] >= -180]['AMT_CREDIT_SUM_OVERDUE'].mean(),
        'bureau_new_credits_6m': (x['DAYS_CREDIT'] >= -180).sum(),
        'bureau_new_credits_12m': (x['DAYS_CREDIT'] >= -365).sum(),

        # Aceleración de deuda
        'bureau_debt_acceleration': (
            x[x['DAYS_CREDIT'] >= -180]['AMT_CREDIT_SUM_DEBT'].mean() -
            x[(x['DAYS_CREDIT'] >= -365) & (x['DAYS_CREDIT'] < -180)]['AMT_CREDIT_SUM_DEBT'].mean()
        ),

        # Cierre de créditos (señal positiva o negativa)
        'bureau_closed_last_year': (
            (x['DAYS_CREDIT'] >= -365) & (x['CREDIT_ACTIVE'] == 'Closed')
        ).sum(),

        # Créditos vendidos (bad sign)
        'bureau_sold_count': (x['CREDIT_ACTIVE'] == 'Sold').sum(),
        'bureau_bad_debt_count': (x['CREDIT_ACTIVE'] == 'Bad debt').sum()
    }), include_groups=False
).reset_index()

# NUEVO: Análisis de bureau_balance detallado
if not bureau_balance.empty:
    bureau_balance_adv = bureau_balance.merge(
        bureau[['SK_ID_BUREAU', 'SK_ID_CURR']],
        on='SK_ID_BUREAU'
    )

    # Contar estados problemáticos por períodos
    bureau_balance_features = bureau_balance_adv.groupby('SK_ID_CURR', group_keys=False).apply(
        lambda x: pd.Series({
            'bureau_dpd_count': (x['STATUS'].isin(['1', '2', '3', '4', '5'])).sum(),
            'bureau_severe_dpd_count': (x['STATUS'].isin(['4', '5'])).sum(),
            'bureau_dpd_ratio': (x['STATUS'].isin(['1', '2', '3', '4', '5'])).mean(),
            'bureau_recent_dpd': (
                (x['MONTHS_BALANCE'] >= -6) &
                (x['STATUS'].isin(['1', '2', '3', '4', '5']))
            ).sum(),
            'bureau_current_status_C': (
                (x['MONTHS_BALANCE'] >= -3) & (x['STATUS'] == 'C')
            ).mean()  # 'C' = closed, señal positiva
        }), include_groups=False
    ).reset_index()
else:
    bureau_balance_features = pd.DataFrame({'SK_ID_CURR': []})

# Merge de todas las features de bureau
bureau_features = (
    bureau_features
    .merge(bureau_type_pivot, on='SK_ID_CURR', how='left')
    .merge(bureau_diversity, on='SK_ID_CURR', how='left')
    .merge(bureau_ratios, on='SK_ID_CURR', how='left')
    .merge(bureau_temporal_advanced, on='SK_ID_CURR', how='left')
)
if not bureau_balance_features.empty:
    bureau_features = bureau_features.merge(bureau_balance_features, on='SK_ID_CURR', how='left')

# ==========================================
# 2. PREVIOUS APPLICATIONS - FEATURES AVANZADAS
# ==========================================
print("🔥 Generando features avanzadas de Previous Applications...")

# Análisis de rechazo/aprobación más granular
prev_advanced = previous.groupby('SK_ID_CURR', group_keys=False).apply(
    lambda x: pd.Series({
        # Tasa de aprobación por período
        'prev_approval_rate_6m': (
            (x['DAYS_DECISION'] >= -180) & (x['NAME_CONTRACT_STATUS'] == 'Approved')
        ).sum() / ((x['DAYS_DECISION'] >= -180).sum() + 1),

        'prev_approval_rate_12m': (
            (x['DAYS_DECISION'] >= -365) & (x['NAME_CONTRACT_STATUS'] == 'Approved')
        ).sum() / ((x['DAYS_DECISION'] >= -365).sum() + 1),

        # Cancelaciones (cliente se arrepiente - señal de inestabilidad)
        'prev_cancelled_count': (x['NAME_CONTRACT_STATUS'] == 'Cancelled').sum(),
        'prev_cancelled_ratio': (x['NAME_CONTRACT_STATUS'] == 'Cancelled').mean(),

        # Análisis de montos solicitados vs aprobados
        'prev_amt_approved_ratio': (
            x[x['NAME_CONTRACT_STATUS'] == 'Approved']['AMT_APPLICATION'].sum() /
            (x['AMT_APPLICATION'].sum() + 1)
        ),

        # Ratio de crédito solicitado vs recibido
        'prev_credit_down_payment_ratio': (
            x['AMT_DOWN_PAYMENT'].sum() / (x['AMT_APPLICATION'].sum() + 1)
        ),

        # Cambio en montos solicitados (¿desesperación?)
        'prev_amt_increasing_trend': (
            x[x['DAYS_DECISION'] >= -365]['AMT_APPLICATION'].mean() /
            (x[x['DAYS_DECISION'] < -365]['AMT_APPLICATION'].mean() + 1)
        ),

        # Productos solicitados
        'prev_product_diversity': x['NAME_CONTRACT_TYPE'].nunique(),
        'prev_revolving_count': (x['NAME_CONTRACT_TYPE'] == 'Revolving loans').sum()
    }), include_groups=False
).reset_index()

# Análisis de ofertas vs aceptación
prev_offer_analysis = previous.groupby('SK_ID_CURR', group_keys=False).apply(
    lambda x: pd.Series({
        'prev_yield_group_high': (x['NAME_YIELD_GROUP'] == 'high').sum(),
        'prev_yield_group_low': (x['NAME_YIELD_GROUP'] == 'low').sum(),
        'prev_goods_category_count': x['NAME_GOODS_CATEGORY'].nunique()
    }), include_groups=False
).reset_index()

prev_features = (
    prev_features
    .merge(prev_advanced, on='SK_ID_CURR', how='left')
    .merge(prev_offer_analysis, on='SK_ID_CURR', how='left')
)

# ==========================================
# 3. INSTALLMENTS - FEATURES AVANZADAS
# ==========================================
print("🔥 Generando features avanzadas de Installments...")

installments_adv = installments.groupby('SK_ID_CURR', group_keys=False).apply(
    lambda x: pd.Series({
        # Pagos parciales vs completos
        'inst_partial_payment_ratio': (
            x['AMT_PAYMENT'] < x['AMT_INSTALMENT']
        ).mean(),

        # Overpayment (paga más de lo debido - señal positiva)
        'inst_overpayment_ratio': (
            x['AMT_PAYMENT'] > x['AMT_INSTALMENT']
        ).mean(),

        # Volatilidad en pagos
        'inst_payment_volatility': x['AMT_PAYMENT'].std() / (x['AMT_PAYMENT'].mean() + 1),

        # Retrasos severos (>30 días)
        'inst_severe_late_ratio': (x['dbd'] > 30).mean(),
        'inst_severe_late_count': (x['dbd'] > 30).sum(),

        # Mejora en comportamiento de pago
        'inst_payment_improvement': (
            x[x['DAYS_INSTALMENT'] >= -365]['late'].mean() -
            x[x['DAYS_INSTALMENT'] < -365]['late'].mean()
        ),

        # Consistencia de pago
        'inst_payment_consistency': 1 / (x['dbd'].std() + 1),

        # Gap máximo entre pagos
        'inst_max_payment_gap': x['dbd'].max() - x['dbd'].min(),

        # Últimos 3 pagos (muy reciente)
        'inst_last_3_late_ratio': x.nlargest(3, 'DAYS_INSTALMENT')['late'].mean() if len(x) >= 3 else 0
    }), include_groups=False
).reset_index()

inst_features = inst_features.merge(installments_adv, on='SK_ID_CURR', how='left')

# ==========================================
# 4. POS_CASH - FEATURES AVANZADAS
# ==========================================
print("🔥 Generando features avanzadas de POS Cash...")

pos_adv = pos_cash.groupby('SK_ID_CURR', group_keys=False).apply(
    lambda x: pd.Series({
        # DPD categories más granulares
        'pos_dpd_0_count': (x['SK_DPD'] == 0).sum(),
        'pos_dpd_30_60_count': ((x['SK_DPD'] > 30) & (x['SK_DPD'] <= 60)).sum(),
        'pos_dpd_60_plus_count': (x['SK_DPD'] > 60).sum(),

        # Evolución de DPD (con manejo seguro)
        'pos_dpd_worsening': int(
            x[x['MONTHS_BALANCE'] >= -3]['SK_DPD'].mean() >
            x[(x['MONTHS_BALANCE'] >= -12) & (x['MONTHS_BALANCE'] < -3)]['SK_DPD'].mean()
        ) if (len(x[x['MONTHS_BALANCE'] >= -3]) > 0 and
              len(x[(x['MONTHS_BALANCE'] >= -12) & (x['MONTHS_BALANCE'] < -3)]) > 0) else 0,

        # Completitud de contratos (finalizados vs activos)
        'pos_completed_contracts': (x['NAME_CONTRACT_STATUS'] == 'Completed').sum(),
        'pos_active_contracts': (x['NAME_CONTRACT_STATUS'] == 'Active').sum(),
        'pos_completion_ratio': (
            (x['NAME_CONTRACT_STATUS'] == 'Completed').sum() /
            (x['SK_ID_PREV'].nunique() + 1)
        ),

        # Volatilidad de DPD
        'pos_dpd_volatility': x['SK_DPD'].std() if len(x) > 1 else 0,
        'pos_dpd_max_ever': x['SK_DPD'].max()
    }), include_groups=False
).reset_index()

pos_features = pos_features.merge(pos_adv, on='SK_ID_CURR', how='left')

# ==========================================
# 5. CREDIT CARD - FEATURES AVANZADAS
# ==========================================
print("🔥 Generando features avanzadas de Credit Card...")

cc_adv = credit_card.groupby('SK_ID_CURR', group_keys=False).apply(
    lambda x: pd.Series({
        # Utilización extrema
        'cc_max_utilization': (
            x['AMT_BALANCE'].max() / (x['AMT_CREDIT_LIMIT_ACTUAL'].max() + 1)
        ),

        # Pagos mínimos (mala señal)
        'cc_min_payment_ratio': (
            x['AMT_PAYMENT_CURRENT'].sum() / (x['AMT_INST_MIN_REGULARITY'].sum() + 1)
        ),

        # Saldo que crece (con manejo seguro)
        'cc_balance_growing': int(
            x[x['MONTHS_BALANCE'] >= -3]['AMT_BALANCE'].mean() >
            x[(x['MONTHS_BALANCE'] >= -12) & (x['MONTHS_BALANCE'] < -3)]['AMT_BALANCE'].mean()
        ) if (len(x[x['MONTHS_BALANCE'] >= -3]) > 0 and
              len(x[(x['MONTHS_BALANCE'] >= -12) & (x['MONTHS_BALANCE'] < -3)]) > 0) else 0,

        # Drawings (retiros de efectivo - mala señal)
        'cc_drawings_atm_ratio': (
            x['AMT_DRAWINGS_ATM_CURRENT'].sum() / (x['AMT_DRAWINGS_CURRENT'].sum() + 1)
        ),
        'cc_drawings_count': x['CNT_DRAWINGS_CURRENT'].sum(),

        # DPD en tarjetas
        'cc_dpd_count': (x['SK_DPD'] > 0).sum(),
        'cc_dpd_recent': x[x['MONTHS_BALANCE'] >= -6]['SK_DPD'].max() if len(x[x['MONTHS_BALANCE'] >= -6]) > 0 else 0,

        # Receivables (dinero pendiente de recibir - complicado)
        'cc_receivable_ratio': (
            x['AMT_RECEIVABLE_PRINCIPAL'].sum() / (x['AMT_BALANCE'].sum() + 1)
        ),

        # Volatilidad de uso
        'cc_usage_volatility': x['AMT_BALANCE'].std() / (x['AMT_BALANCE'].mean() + 1) if len(x) > 1 else 0
    }), include_groups=False
).reset_index()

cc_features = cc_features.merge(cc_adv, on='SK_ID_CURR', how='left')

# ==========================================
# 6. CROSS-TABLE FEATURES AVANZADAS
# ==========================================
print("🔥 Generando features combinadas avanzadas...")

# Merge intermedio para crear cross-features
df_train_temp = (
    app_train
    .merge(bureau_features, on="SK_ID_CURR", how="left")
    .merge(prev_features, on="SK_ID_CURR", how="left")
    .merge(inst_features, on="SK_ID_CURR", how="left")
    .merge(pos_features, on="SK_ID_CURR", how="left")
    .merge(cc_features, on="SK_ID_CURR", how="left")
)

# Features de interacción críticas
cross_features = pd.DataFrame({'SK_ID_CURR': df_train_temp['SK_ID_CURR']})

# Función auxiliar para verificar si columna existe
def safe_get_column(df, col_name, default=0):
    if col_name in df.columns:
        return df[col_name].fillna(default)
    else:
        return pd.Series([default] * len(df), index=df.index)

# Carga de deuda total vs ingresos
cross_features['total_debt_to_income'] = (
    (
        safe_get_column(df_train_temp, 'bureau_weighted_debt') +
        safe_get_column(df_train_temp, 'cc_balance_mean')
    ) / (df_train_temp['AMT_INCOME_TOTAL'] + 1)
)

# Score de deterioro combinado mejorado (usando solo columnas que existen)
cross_features['deterioration_composite_score'] = (
    safe_get_column(df_train_temp, 'inst_late_deterioration') * 2 +
    safe_get_column(df_train_temp, 'cc_balance_growing') * 1.5 +
    (safe_get_column(df_train_temp, 'bureau_recent_vs_old_debt', 1) - 1) +
    safe_get_column(df_train_temp, 'pos_dpd_acceleration')
)

# Ratio de aplicaciones rechazadas recientes (señal fuerte)
cross_features['recent_rejection_intensity'] = (
    safe_get_column(df_train_temp, 'prev_recent_refused_count') /
    (safe_get_column(df_train_temp, 'prev_recent_refused_count') +
     safe_get_column(df_train_temp, 'prev_recent_approved_count') + 1)
)

# Capacidad de pago mensual
cross_features['monthly_payment_capacity'] = (
    (df_train_temp['AMT_INCOME_TOTAL'] / 12) -
    (safe_get_column(df_train_temp, 'AMT_ANNUITY') +
     safe_get_column(df_train_temp, 'inst_amt_paid_1y') / 12)
)

# Red flags combinadas
cross_features['red_flags_count'] = (
    (safe_get_column(df_train_temp, 'bureau_bad_debt_count') > 0).astype(int) +
    (safe_get_column(df_train_temp, 'bureau_sold_count') > 0).astype(int) +
    (safe_get_column(df_train_temp, 'prev_cancelled_ratio') > 0.3).astype(int) +
    (safe_get_column(df_train_temp, 'inst_severe_late_ratio') > 0.2).astype(int) +
    (safe_get_column(df_train_temp, 'pos_dpd_60_plus_count') > 0).astype(int) +
    (safe_get_column(df_train_temp, 'cc_dpd_recent') > 30).astype(int)
)

# Positive signals combinadas
cross_features['positive_signals_count'] = (
    (safe_get_column(df_train_temp, 'inst_overpayment_ratio') > 0).astype(int) +
    (safe_get_column(df_train_temp, 'bureau_current_status_C') > 0.5).astype(int) +
    (safe_get_column(df_train_temp, 'pos_completion_ratio') > 0.5).astype(int) +
    (safe_get_column(df_train_temp, 'prev_approval_rate_12m') > 0.7).astype(int)
)

# Credit mix score (diversidad = estabilidad)
cross_features['credit_mix_score'] = (
    safe_get_column(df_train_temp, 'bureau_credit_diversity') +
    safe_get_column(df_train_temp, 'prev_product_diversity') / 5
)

# Engagement score (cuánto usa el sistema financiero)
cross_features['financial_engagement_score'] = (
    np.log1p(safe_get_column(df_train_temp, 'bureau_loans')) +
    np.log1p(safe_get_column(df_train_temp, 'prev_apps')) +
    np.log1p(safe_get_column(df_train_temp, 'cc_loans'))
)

# Merge final
df_train = df_train_temp.merge(cross_features, on='SK_ID_CURR', how='left')

# Rellenar NaN en las nuevas features
df_train = df_train.fillna(0)

# ==========================================
# LIMPIEZA DE TIPOS DE DATOS PARA PARQUET
# ==========================================
print("🔧 Limpiando tipos de datos para guardar...")

# Convertir columnas object a string o numérico según corresponda
for col in df_train.columns:
    if df_train[col].dtype == 'object':
        # Intentar convertir a numérico primero
        try:
            df_train[col] = pd.to_numeric(df_train[col], errors='ignore')
        except:
            pass

        # Si sigue siendo object, convertir a string
        if df_train[col].dtype == 'object':
            df_train[col] = df_train[col].astype(str)

# Convertir booleanos a int
bool_columns = df_train.select_dtypes(include=['bool']).columns
if len(bool_columns) > 0:
    df_train[bool_columns] = df_train[bool_columns].astype(int)

# ==========================================
# GUARDAR RESULTADO
# ==========================================
df_train.to_parquet("../data/interim/train_final_advanced_features2.parquet", engine='pyarrow', index=False)

print("\n" + "="*60)
print("✅ FEATURES AVANZADAS COMPLETADAS")
print("="*60)
print(f"Total de columnas: {df_train.shape[1]}")
print(f"Total de filas: {df_train.shape[0]}")
print(f"\n📊 NUEVAS FEATURES AÑADIDAS POR CATEGORÍA:")
print(f"\n🏦 BUREAU ({len([c for c in df_train.columns if 'bureau' in c])} features):")
print(f"  - Diversidad de créditos (entropy)")
print(f"  - Ratios deuda/crédito críticos")
print(f"  - Análisis de estados DPD detallado")
print(f"  - Créditos vendidos/bad debt")
print(f"  - Aceleración de deuda")
print(f"\n📝 PREVIOUS APPS ({len([c for c in df_train.columns if 'prev' in c])} features):")
print(f"  - Tasas de aprobación por período")
print(f"  - Cancelaciones del cliente")
print(f"  - Tendencias en montos solicitados")
print(f"  - Análisis de productos solicitados")
print(f"\n💳 INSTALLMENTS ({len([c for c in df_train.columns if 'inst' in c])} features):")
print(f"  - Pagos parciales vs overpayments")
print(f"  - Volatilidad y consistencia")
print(f"  - Retrasos severos (>30 días)")
print(f"  - Mejora en comportamiento")
print(f"\n🏪 POS CASH ({len([c for c in df_train.columns if 'pos' in c])} features):")
print(f"  - DPD categories granulares")
print(f"  - Ratio de completitud")
print(f"  - Volatilidad DPD")
print(f"\n💰 CREDIT CARD ({len([c for c in df_train.columns if 'cc' in c])} features):")
print(f"  - Utilización extrema")
print(f"  - Drawings y retiros ATM")
print(f"  - Crecimiento de saldo")
print(f"  - Volatilidad de uso")
print(f"\n🔗 CROSS-FEATURES (12 features combinadas):")
print(f"  - total_debt_to_income")
print(f"  - deterioration_composite_score")
print(f"  - recent_rejection_intensity ⚡")
print(f"  - monthly_payment_capacity")
print(f"  - red_flags_count (6 señales negativas)")
print(f"  - positive_signals_count (4 señales positivas)")
print(f"  - credit_mix_score")
print(f"  - financial_engagement_score")
print(f"\n💾 Archivo guardado: ../data/interim/train_final_advanced_features2.parquet")

# Mostrar las features más prometedoras
print(f"\n🎯 TOP FEATURES PROMETEDORAS PARA MEJORAR AUC:")
print(f"  1. recent_rejection_intensity (rechazos recientes)")
print(f"  2. red_flags_count (múltiples señales de riesgo)")
print(f"  3. bureau_bad_debt_count (deudas incobrables)")
print(f"  4. inst_severe_late_ratio (retrasos >30 días)")
print(f"  5. deterioration_composite_score (empeoramiento)")
print(f"  6. pos_dpd_60_plus_count (morosidad severa)")
print(f"  7. monthly_payment_capacity (capacidad real)")
print(f"  8. prev_cancelled_ratio (inestabilidad)")

print("\n" + "="*60)
print("Ejecuta tu modelo con estas nuevas features para ver mejoras!")
print("="*60)

🔥 Generando features avanzadas de Bureau...
🔥 Generando features avanzadas de Previous Applications...
🔥 Generando features avanzadas de Installments...
🔥 Generando features avanzadas de POS Cash...
🔥 Generando features avanzadas de Credit Card...
🔥 Generando features combinadas avanzadas...
🔧 Limpiando tipos de datos para guardar...


C:\Users\usuario\AppData\Local\Temp\ipykernel_1832\2699563398.py:389: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_train[col] = pd.to_numeric(df_train[col], errors='ignore')
C:\Users\usuario\AppData\Local\Temp\ipykernel_1832\2699563398.py:389: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_train[col] = pd.to_numeric(df_train[col], errors='ignore')
C:\Users\usuario\AppData\Local\Temp\ipykernel_1832\2699563398.py:389: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_train[col] = pd.to_numeric(df_train[col], errors='ignore')
C:\Users\usuario\AppData\Local\Temp\ipykernel_1832\2699563398.py:389: FutureWarning: errors='ignore' is deprecated 


✅ FEATURES AVANZADAS COMPLETADAS
Total de columnas: 191
Total de filas: 307511

📊 NUEVAS FEATURES AÑADIDAS POR CATEGORÍA:

🏦 BUREAU (46 features):
  - Diversidad de créditos (entropy)
  - Ratios deuda/crédito críticos
  - Análisis de estados DPD detallado
  - Créditos vendidos/bad debt
  - Aceleración de deuda

📝 PREVIOUS APPS (22 features):
  - Tasas de aprobación por período
  - Cancelaciones del cliente
  - Tendencias en montos solicitados
  - Análisis de productos solicitados

💳 INSTALLMENTS (20 features):
  - Pagos parciales vs overpayments
  - Volatilidad y consistencia
  - Retrasos severos (>30 días)
  - Mejora en comportamiento

🏪 POS CASH (27 features):
  - DPD categories granulares
  - Ratio de completitud
  - Volatilidad DPD

💰 CREDIT CARD (19 features):
  - Utilización extrema
  - Drawings y retiros ATM
  - Crecimiento de saldo
  - Volatilidad de uso

🔗 CROSS-FEATURES (12 features combinadas):
  - total_debt_to_income
  - deterioration_composite_score
  - recent_rejection_